# Zero to Hero

## 1.07 Aplicando el arbol

Se aplicará el modelo generado por rpart a los datos nuevos y se genera la salida para Kaggle

In [ ]:
rm(list = ls())
gc()

library("data.table") # cargo la libreria  data.table
library("rpart") # cargo la libreria  rpart
library("rpart.plot")

options(repr.plot.width = 20, repr.plot.height = 10)
setwd("~/buckets/b1/")


In [ ]:
dataset <- fread("~/datasets/vivencial_dataset_pequeno.csv") # cargo el dataset #cargo el dataset


In [ ]:
dtrain <- dataset[foto_mes == 202107]
dapply <- dataset[foto_mes == 202109]


In [ ]:
# genero el modelo
modelo <- rpart::rpart(
    formula = "clase_ternaria ~ .",
    data = dtrain,
    xval = 0,
    cp = -1,
    maxdepth = 2
)


Aplico el modelo a los datos dapply  pidiendo que me devuelva probabildades

In [ ]:
prediccion <- predict(modelo, dapply, type = "prob")


Analizo que queda en prediccion

In [ ]:
head(prediccion)


Prediccion es una matriz; me interesa la columna "BAJA+2"  que es la probabilidad que modelo asigna a cada registro de dapply

In [ ]:
prob_baja2 <- prediccion[, "BAJA+2"]


In [ ]:
head(prob_baja2)


prob_baja2 es el vector de probabilidades

In [ ]:
length(prob_baja2)
nrow(dapply)


Ahora decido si envio el estimulo o no
<br>
si prob( BAJA+2) > 0.025  envio  el estímulo

In [ ]:
Predicted <- ifelse(prob_baja2 > 0.025, 1, 0)


In [ ]:
head(Predicted)


In [ ]:
hist(Predicted)


creo una tabla con  lo que voy a enviar a Kaggle

In [ ]:
entrega <- as.data.table(list("numero_de_cliente" = dapply$numero_de_cliente, "Predicted" = Predicted))


In [ ]:
head(entrega)


In [ ]:
entrega[, .N, Predicted]


Esto significa que se enviaran 4979 estímulos

In [ ]:
# genero el archivo para Kaggle
# creo la carpeta donde va el experimento
dir.create("./exp/", showWarnings = FALSE)
dir.create("./exp/ZH2017/", showWarnings = FALSE)

fwrite(entrega,
        file = "./exp/ZH2017/para_Kaggle_0107.csv",
        sep = ","
)
